# Machine generated Text Detection

Description of the project

## Baseline:

In [1]:

import pandas as pd
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer, set_seed
import os
from transformers import BertModel
from sklearn.model_selection import train_test_split
import torch
from scipy.special import softmax
import logging
from transformers import BertTokenizer
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Check if GPU is available
print("GPU available:", torch.cuda.is_available())

# Print GPU name
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

GPU available: True
GPU: NVIDIA A100-SXM4-40GB


### Baseline Model:

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import logging
import os

# Read data and split into train/validation/test
def get_data(train_path, test_path, random_seed):
    """
    Function to read dataframe with columns.
    """
    train_df = pd.read_json(train_path, lines=True)
    test_df = pd.read_json(test_path, lines=True)

    train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=random_seed)
    return train_df, val_df, test_df

# Preprocess text data
def preprocess_function(examples):
    return examples["text"]

# Train Naive Bayes model
def fine_tune(train_df, valid_df):
    # Extract text and labels
    X_train = train_df['text']
    y_train = train_df['label']
    X_valid = valid_df['text']
    y_valid = valid_df['label']

    # Vectorize text data
    vectorizer = CountVectorizer()
    X_train_vec = vectorizer.fit_transform(X_train)
    X_valid_vec = vectorizer.transform(X_valid)

    # Initialize Naive Bayes model
    model = MultinomialNB()
    model.fit(X_train_vec, y_train)

    # Evaluate the model on the validation set
    y_pred = model.predict(X_valid_vec)
    report = classification_report(y_valid, y_pred)
    logging.info(f"Validation Classification Report:\n{report}")

    return model, vectorizer

# Test Naive Bayes model
def test(test_df, model, vectorizer):
    X_test = test_df['text']
    y_test = test_df['label']

    # Vectorize test data
    X_test_vec = vectorizer.transform(X_test)

    # Make predictions
    y_pred = model.predict(X_test_vec)

    # Get classification report
    report = classification_report(y_test, y_pred)
    logging.info(f"Test Classification Report:\n{report}")

    return report, y_pred

# Main script
random_seed = 0
train_path = 'subtaskA_train_monolingual.jsonl'
test_path = 'subtaskA_monolingual_gold.jsonl'
prediction_path = 'baseline_Results.jsonl'

if not os.path.exists(train_path):
    logging.error(f"File doesn't exist: {train_path}")
    raise ValueError(f"File doesn't exist: {train_path}")

if not os.path.exists(test_path):
    logging.error(f"File doesn't exist: {test_path}")
    raise ValueError(f"File doesn't exist: {test_path}")

# Get data for train/dev/test sets
train_df, valid_df, test_df = get_data(train_path, test_path, random_seed)

# Train the Naive Bayes model
model, vectorizer = fine_tune(train_df, valid_df)

# Test the Naive Bayes model
results, predictions = test(test_df, model, vectorizer)

# Save predictions to file
predictions_df = pd.DataFrame({'id': test_df['id'], 'label': predictions})
predictions_df.to_json(prediction_path, lines=True, orient='records')


### Baseline Evaluation:

In [8]:
!python3 ./scorer.py --gold_file_path="subtaskA_monolingual_gold.jsonl" --pred_file_path="./baseline_Results.jsonl"

INFO : Prediction file format is correct
INFO : macro-F1=0.84967	micro-F1=0.84967	accuracy=0.84967


## Deep Learning:

### Deep Learning Based Model:

In [9]:
import pandas as pd
import torch
from transformers import RobertaTokenizerFast
from sklearn.model_selection import train_test_split

# Load data
train_path = 'subtaskA_train_monolingual.jsonl'
test_path = 'subtaskA_monolingual_gold.jsonl'

train_df = pd.read_json(train_path, lines=True)
test_df = pd.read_json(test_path, lines=True)

train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=42)

# Initialize fast tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# Set a smaller max length if possible
MAX_LEN = 256  # Adjust based on your data; 128 or 256 is usually enough

# Function to tokenize in batches
def tokenize_batch(texts, batch_size=1000):
    # Tokenize in batches to reduce memory usage
    encodings = tokenizer(
        texts, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt"
    )
    return encodings

# Tokenize train, validation, and test data in batches
batch_size = 1000  # You can adjust this batch size based on available memory

# Tokenizing training data in batches
train_encodings = {'input_ids': [], 'attention_mask': []}
for i in range(0, len(train_df), batch_size):
    batch = train_df['text'].iloc[i:i + batch_size].tolist()
    batch_encodings = tokenize_batch(batch)
    train_encodings['input_ids'].append(batch_encodings['input_ids'])
    train_encodings['attention_mask'].append(batch_encodings['attention_mask'])

train_input_ids = torch.cat(train_encodings['input_ids'], dim=0)
train_attention_masks = torch.cat(train_encodings['attention_mask'], dim=0)

# Tokenizing validation data in batches
val_encodings = {'input_ids': [], 'attention_mask': []}
for i in range(0, len(val_df), batch_size):
    batch = val_df['text'].iloc[i:i + batch_size].tolist()
    batch_encodings = tokenize_batch(batch)
    val_encodings['input_ids'].append(batch_encodings['input_ids'])
    val_encodings['attention_mask'].append(batch_encodings['attention_mask'])

val_input_ids = torch.cat(val_encodings['input_ids'], dim=0)
val_attention_masks = torch.cat(val_encodings['attention_mask'], dim=0)

# Tokenizing test data in batches
test_encodings = {'input_ids': [], 'attention_mask': []}
for i in range(0, len(test_df), batch_size):
    batch = test_df['text'].iloc[i:i + batch_size].tolist()
    batch_encodings = tokenize_batch(batch)
    test_encodings['input_ids'].append(batch_encodings['input_ids'])
    test_encodings['attention_mask'].append(batch_encodings['attention_mask'])

test_input_ids = torch.cat(test_encodings['input_ids'], dim=0)
test_attention_masks = torch.cat(test_encodings['attention_mask'], dim=0)

print("✅ Tokenization Done! Training can start.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

✅ Tokenization Done! Training can start.


In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import RobertaModel, RobertaTokenizer
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from torch.optim import AdamW
import torch.nn.functional as F

# Define model architecture
class SimpleMultiLayerNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.3):
        super(SimpleMultiLayerNN, self).__init__()

        # Use a smaller pre-trained BERT model (Roberta)
        self.bert = RobertaModel.from_pretrained('roberta-base')

        for param in self.bert.parameters():
          param.requires_grad = False

        # Unfreeze last 2 layers
        # for param in self.bert.encoder.layer[-1:].parameters():
        #     param.requires_grad = True

        # First hidden layer
        self.hidden1 = nn.Linear(input_size, hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)

        # Output layer
        self.output = nn.Linear(hidden_size, output_size)

    def forward(self, input_ids, attention_mask):
        # Get Roberta embeddings
        bert_output = self.bert(input_ids, attention_mask=attention_mask)
        last_hidden_state = bert_output.last_hidden_state  # [batch_size, seq_len, 768]

        # Weighted Pooling: Use attention_mask to weight token embeddings
        attention_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size())
        weighted_sum = torch.sum(last_hidden_state * attention_mask_expanded, 1)

        # Normalize pooled output by the attention mask
        pooled_output = weighted_sum / attention_mask_expanded.sum(1)  # [batch_size, 768]

        # First hidden layer + ReLU + Dropout
        x = F.relu(self.hidden1(pooled_output))
        x = self.dropout1(x)

        # Output layer (logits for multi-class classification)
        x = self.output(x)

        return x

# Example usage
input_size = 768  # Output size of Roberta's hidden layers
hidden_size = 128  # Size of hidden layers
output_size = 2  # Number of output classes (adjust for binary/multi-class classification)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available

# Initialize the model
model = SimpleMultiLayerNN(input_size, hidden_size, output_size)
model.to(device)

# Setup loss function and optimizer with weight decay (L2 Regularization)
loss_fn = nn.CrossEntropyLoss()

optimizer = AdamW([
    {'params': model.bert.encoder.layer[:3].parameters(), 'lr': 5e-6},  # Lower layers
    {'params': model.bert.encoder.layer[3:].parameters(), 'lr': 5e-5},  # Higher layers
    {'params': model.output.parameters(), 'lr': 1e-4}  # Classifier head
], weight_decay=0.01)  # Adding weight decay for regularization

# Learning rate scheduler for gradual learning rate decay
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5, verbose=True)

# Prepare DataLoader for training
train_dataset = TensorDataset(train_input_ids, train_attention_masks, torch.tensor(train_df['label'].values))
val_dataset = TensorDataset(val_input_ids, val_attention_masks, torch.tensor(val_df['label'].values))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)  # Using smaller batch size for better generalization
val_loader = DataLoader(val_dataset, batch_size=64, num_workers=2)

# Training Loop
epochs = 3  # You can change this
for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{epochs}"):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Prediction
        preds = torch.argmax(outputs, dim=-1)
        correct_predictions += (preds == labels).sum().item()
        total_predictions += labels.size(0)

    avg_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / total_predictions
    print(f"Epoch {epoch+1} - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

    # Validation Step
    model.eval()
    val_loss = 0
    correct_predictions = 0
    total_predictions = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Validating Epoch {epoch+1}/{epochs}"):
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs, labels)
            val_loss += loss.item()

            preds = torch.argmax(outputs, dim=-1)
            correct_predictions += (preds == labels).sum().item()
            total_predictions += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = correct_predictions / total_predictions
    print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Update learning rate based on validation loss
    scheduler.step(val_loss)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training Epoch 1/3: 100%|██████████| 1497/1497 [04:29<00:00,  5.56it/s]


Epoch 1 - Loss: 0.6848, Accuracy: 0.5596


Validating Epoch 1/3: 100%|██████████| 375/375 [01:05<00:00,  5.71it/s]


Validation Loss: 0.6732, Validation Accuracy: 0.6499


Training Epoch 2/3: 100%|██████████| 1497/1497 [04:28<00:00,  5.57it/s]


...

Validation Loss: 0.6551, Validation Accuracy: 0.7543


Training Epoch 3/3: 100%|██████████| 1497/1497 [04:28<00:00,  5.57it/s]


Epoch 3 - Loss: 0.6495, Accuracy: 0.6716


Validating Epoch 3/3: 100%|██████████| 375/375 [01:05<00:00,  5.72it/s]

Validation Loss: 0.6398, Validation Accuracy: 0.7731


In [32]:
def test(test_df, model, device, batch_size=16):
    model.eval()
    predictions = []
    true_labels = test_df['label'].values
    with torch.no_grad():
        for i in tqdm(range(0, len(test_df), batch_size), desc="Testing"):
            # Move data to the same device as the model
            batch_input_ids = test_input_ids[i:i+batch_size].to(device)
            batch_attention_mask = test_attention_masks[i:i+batch_size].to(device)
            batch_labels = true_labels[i:i+batch_size]

            # Make sure the model is on the correct device
            model = model.to(device)

            output = model(batch_input_ids, batch_attention_mask)
            preds = torch.argmax(output, dim=-1).cpu().numpy()
            predictions.extend(preds)

    # report = classification_report(true_labels, predictions)
    # print(f"Test Classification Report:\n{report}")
    return predictions

# Make sure to set your device (CPU or CUDA)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Run test and get predictions
predictions = test(test_df, model, device)

# Save predictions to file
predictions_df = pd.DataFrame({'id': test_df['id'], 'label': predictions})

Testing: 100%|██████████| 2142/2142 [01:46<00:00, 20.12it/s]


### Deep Learning Evaluation:

In [33]:
predictions_df.to_json("baseline_Results_deep.jsonl", lines=True, orient='records')

In [34]:
!python3 ./scorer.py --gold_file_path="subtaskA_monolingual_gold.jsonl" --pred_file_path="./baseline_Results_deep.jsonl"

INFO : Prediction file format is correct
INFO : macro-F1=0.92747	micro-F1=0.92752	accuracy=0.92752


## LLM:

### LLM Based Model:

### LLM Evaluation: